<a href="https://colab.research.google.com/github/scumulder/Machine-Learning-Spring-2019-UCF/blob/master/Homework%202/CNN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**About this CNN:**


---




---




**Loading the CIFAR10 data set:**


In [0]:
from keras.datasets import cifar10
(train_images_original, train_labels_original), (test_images, test_labels) = cifar10.load_data()

#X_train = train_images
#Y_train = train_labels

#X_test = test_images
#Y_test = test_labels

Using TensorFlow backend.


170500096/170498071 [==============================] - 31s 0us/step


**Split Training Data into Training Set and Validation Set:**

20% of the training data will be used to form the validation set and the remaining 80% will be the new training set

In [0]:
import sklearn.model_selection

train_images, val_images, train_labels, val_labels = sklearn.model_selection.train_test_split(train_images_original, train_labels_original, 
                                                                                              test_size=0.20, random_state=42)
print('Training image shape:', train_images.shape)
print('Validation image shape:', val_images.shape)

Training image shape: (40000, 32, 32, 3)
Validation image shape: (10000, 32, 32, 3)


**Prepare the training and validation image data:**

In [0]:
train_images = train_images.astype('float32')/ 255
val_images = val_images.astype('float32')/ 255

**Prepare the labels:**

    airplane : 0     automobile : 1    bird : 2    cat : 3          deer : 4       dog : 5    frog : 6         horse : 7         ship : 8          truck : 9

In [0]:
import numpy as np
from keras.utils.np_utils import to_categorical   

train_labels = to_categorical(train_labels)
val_labels = to_categorical(val_labels)

**Convolutional Neural Network Architecture:**

In [0]:
from keras import models
from keras import layers

batch_size = 256
num_classes = 10
epochs = 25

network = models.Sequential()

network.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
network.add(layers.Conv2D(64, (3, 3), activation='relu'))
network.add(layers.MaxPooling2D(pool_size = (2, 2)))

network.add(layers.Conv2D(128, (3, 3), activation='relu'))
network.add(layers.MaxPooling2D(pool_size = (2, 2)))

#network.add(layers.Dropout(0.50)) #0.25 initially
network.add(layers.Flatten())

network.add(layers.Dense(128, activation='relu'))
network.add(layers.Dropout(0.50))
network.add(layers.Dense(num_classes, activation='softmax'))
network.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               589952    
__________

**Initialize the Image Data Generator for Data Augmentation:**

In [0]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator( rotation_range=15,
                 width_shift_range=0.1, height_shift_range=0.1,
                 horizontal_flip=True)
datagen.fit(train_images)

**Compile the network:**

In [0]:
network.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

**Train the network:**

In [0]:
network.fit_generator(datagen.flow(train_images, train_labels, batch_size=batch_size),steps_per_epoch=100, epochs=epochs, verbose=1)

Epoch 1/25
100/100 [==============================] - 13s 134ms/step - loss: 2.0693 - acc: 0.2421
Epoch 2/25
100/100 [==============================] - 14s 138ms/step - loss: 1.7953 - acc: 0.3561
Epoch 3/25
100/100 [==============================] - 14s 138ms/step - loss: 1.6517 - acc: 0.4101
Epoch 4/25
100/100 [==============================] - 14s 137ms/step - loss: 1.5515 - acc: 0.4480
Epoch 5/25
100/100 [==============================] - 14s 138ms/step - loss: 1.4445 - acc: 0.4849
Epoch 6/25
100/100 [==============================] - 14s 137ms/step - loss: 1.3871 - acc: 0.5062
Epoch 7/25
100/100 [==============================] - 14s 138ms/step - loss: 1.3334 - acc: 0.5304
Epoch 8/25
100/100 [==============================] - 14s 138ms/step - loss: 1.2514 - acc: 0.5632
Epoch 9/25
100/100 [==============================] - 14s 137ms/step - loss: 1.2310 - acc: 0.5669
Epoch 10/25
100/100 [==============================] - 14s 138ms/step - loss: 1.1797 - acc: 0.5861
Epoch 11/25
100/100

**Evaluate the network on the Validation Set:**

In [0]:
score = network.evaluate(val_images, val_labels, verbose=0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1]*100,"%")

Validation loss: 0.7746413391113282
Validation accuracy: 73.11 %
